In [2]:
print("Ok")

Ok


In [3]:
%pwd

'e:\\Projects\\Medical-Chatbot\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'e:\\Projects\\Medical-Chatbot'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
#extracted_data

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [12]:
#text_chunks

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\shiva\AppData\Local\Temp\ipykernel_4140\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\shiva\.conda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
query_result = embeddings.embed_query("Hello World")
print("Length:", len(query_result))

Length: 384


In [ ]:
#query_result

[-0.034477200359106064,
 0.031023219227790833,
 0.0067349993623793125,
 0.02610897459089756,
 -0.03936201333999634,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441470701247454,
 -0.04745051637291908,
 0.014758859761059284,
 0.07087533175945282,
 0.05552757531404495,
 0.019193289801478386,
 -0.026251329109072685,
 -0.010109508410096169,
 -0.026940548792481422,
 0.022307483479380608,
 -0.022226618602871895,
 -0.14969265460968018,
 -0.01749304309487343,
 0.007676269859075546,
 0.05435226485133171,
 0.003254482988268137,
 0.03172600269317627,
 -0.0846213772892952,
 -0.02940598875284195,
 0.051595672965049744,
 0.04812409356236458,
 -0.003314794274047017,
 -0.05827922374010086,
 0.041969284415245056,
 0.022210709750652313,
 0.1281888633966446,
 -0.022338956594467163,
 -0.011656241491436958,
 0.06292833387851715,
 -0.032876305282115936,
 -0.09122610837221146,
 -0.03117538057267666,
 0.05269957706332207,
 0.047034792602062225,
 -0.08420304954051971,
 -0.0300561785697937,
 -0.020744718

In [39]:
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
import os
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key= PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,  # Dimension of the embeddings
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-ubczn9p.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [41]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY

In [22]:
#Embed each chunk and upsert the embeddings into Pinecone
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)


In [23]:
#Load the existing index
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [24]:
docsearch

In [25]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [26]:
retrieved_docs = retriever.invoke("What is Acne", k=3)

In [27]:
retrieved_docs

[Document(id='f89c6abf-8737-42e5-9c3b-8167dbc1ce3a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='9401527d-1689-4dd4-bb61-1745ddd75ca3', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 37.0, 'page_label': '38', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteri

In [28]:
#from langchain_openai import OpenAI
#llm = OpenAI(temperature=0.4, max_tokens=500)

In [42]:
# Initialize LLaMA-3-70B via Hugging Face Inference API
from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    endpoint_url="https://api-inference.huggingface.co/models/meta-llama/Llama-3-70b",
    huggingfacehub_api_token=os.environ.get("HUGGINGFACEHUB_API_TOKEN"),
    temperature=0.4, 
    max_new_tokens=512 
)

In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for Question-Answering tasks."
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know."
    "Do not try to make up an answer."
    "Use three sentences maximum and be concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [44]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response['answer'])

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://api-inference.huggingface.co/models/meta-llama/Llama-3-70b (Request ID: Root=1-685d32a9-2a7f81ed0f31f7c86baf7a9f;4eeb0595-e335-4683-b07e-5bcd44af18c5)

Invalid username or password.